# Conclusions

* Start with some random locs in input space. use dpk.utils.keypoints.draw_keypoints to draw kps/confmaps, as done in tfdatagen. 
* This draw confmaps slightly-wrongly-downscaled per the usual bug/issue. If you extract centroid centers via heatmap, and do a naive scale by x4, this matches the inputs, except near edges where the simple centroid-center procedure fails due to edge effects.
* The confmaps so generated (ie slightly offset) are used for training. In an ideal world, the trained network will output precisely the same confmap, given the same input.
* DPK prediction is done via the SubpixelMaxima2D layer. We confirm that this layer "undoes" the slightly-offset downscaling appropriately by simply feeding in the training confmap and confirming that the predicted locations (in the input space) match the original locations. Ie, the Subpixel layer has gone from predicted heatmaps at the downscaled/network output resolution => raw/input resolution with the same simple/naive up/downscaling.
* Tldr, taking raw keypts => generating confmaps => feed thru SubpixelMaxima basically returns the same raw keypts, EXCEPT for pts near the edge (say ~within 6px in input space), which seems reasonable. Our data typically should not have labels so close to the edges.
* Our simple 1-cluster heatmap/centroid subpx extraction matches the DPK SubpixelMaxima layer, except near the edges where it does a bit worse.
* The purpose of the various input parameters to SubpixelMaxima is not clear. Changing some of them did not change the results.
* The 'regular' dpk Maxima2D is equivalent to doing an argmax 

In [ ]:
import heatmap
import numpy as np
import matplotlib.pyplot as plt
from deepposekit.models.backend.backend import find_subpixel_maxima as fsm
import PoseTools as pt
from easydict import EasyDict as edict
from deepposekit.models.layers.subpixel import SubpixelMaxima2D
from deepposekit.models.layers.convolutional import Maxima2D

import tensorflow as tf
import random

import deepposekit as dpk
from deepposekit.utils.image import n_downsample
from deepposekit.utils.image import largest_factor


In [ ]:
PICF = '/dat0/apt/cache/leap_dset/dpksdn/view_0/dpkorig_test1/conf.pickle'

pic = pt.pickle_load(PICF)

sdnconf = edict(pic['sdn'])
output_shape = sdnconf.output_shape
keypoints_shape = sdnconf.keypoints_shape
downsample_factor = sdnconf.downsample_factor
output_sigma = sdnconf.output_sigma

In [ ]:
kernel_size = np.min(output_shape)
kernel_size = (kernel_size // largest_factor(kernel_size)) + 1
sigma = output_sigma

In [ ]:
print(output_shape, keypoints_shape, downsample_factor, output_sigma,
     kernel_size)

# Start with random keypoints 

In [ ]:
RAW_HT = 192
RAW_WH = 192
kps = np.random.rand(*keypoints_shape)*RAW_HT
kps = kps.round(2)  # for easier eyeballing
#RAW_KP = np.array([[13.3333, 21.2]])
#kptile = np.tile(RAW_KP,(32,1))

In [ ]:
kps.shape, kps

# Raw confmaps

In [ ]:
hm2 = dpk.utils.keypoints.draw_keypoints(kps, 
                                         RAW_HT, RAW_WH, 
                                         output_shape, 
                                         output_sigma)
# note, this draw method uses the slightly-off downscaling

In [ ]:
hm2.shape

In [ ]:
plt.imshow(hm2[...,-12])
plt.colorbar()

In [ ]:
# slightly-off downscaling
kps/4

In [ ]:
hmmu, hmam = heatmap.get_weighted_centroids_with_argmax(
    hm2[np.newaxis,...], nclustermax=1)

In [ ]:
# confirms that the drawn confmaps are slightly off in downscaled space
hmmu, hmam  

In [ ]:
hmmu*4, hmam*4

In [ ]:
hm = hm2

In [ ]:
hm.dtype

In [ ]:
hm = hm.astype(np.float32)

In [ ]:
hm = hm[np.newaxis,...]

In [ ]:
hm.dtype, hm.shape

In [ ]:
print(kernel_size, sigma, keypoints_shape, downsample_factor,)

In [ ]:
keypoints = SubpixelMaxima2D(
                kernel_size,
                sigma,
                upsample_factor=100,
                index=keypoints_shape[0],
                coordinate_scale=2 ** downsample_factor,
                confidence_scale=255.0,
            )(hm)

In [ ]:
kppred = tf.keras.backend.eval(keypoints)

In [ ]:
keypoints_crude = Maxima2D(
                index=keypoints_shape[0],
                coordinate_scale=2 ** downsample_factor,
                confidence_scale=255.0,
            )(hm) 

In [ ]:
kppred_crude = tf.keras.backend.eval(keypoints_crude)

In [ ]:
kppred.shape, kppred_crude.shape

In [ ]:
kps2=kppred[0,:,:2]  # 3rd col is confidence

In [ ]:
kps2_crude = kppred_crude[0,:,:2]

In [ ]:
kps2

In [ ]:
kps2_crude

In [ ]:
np.allclose(hmam.astype(np.float32)*4, kps2_crude)

In [ ]:
d=kps2-kps
tf = np.abs(d)>.05

In [ ]:
kps.shape

In [ ]:
tf

In [ ]:
tfrow=np.any(tf,axis=1)
np.nonzero(tfrow)

In [ ]:
kps[tf], kps2[tf], hmmu[0,tf]*4, d[tf]

In [ ]:
row=6; kps[row,:], kps2[row,:], d[row,:]

### Messing Around

In [ ]:
locssz = (1,) + tuple(keypoints_shape)
locs = np.zeros(locssz)
locs[0,:,0] = 13.33333
locs[0,:,1] = 21.2
hm = heatmap.create_label_hmap(locs, output_shape, sigma)

In [ ]:
plt.imshow(hm[0,...,3])

In [ ]:
tf.keras.backend.eval(keypoints)

In [ ]:
RAW_KP


In [ ]:
kp2 = SubpixelMaxima2D(
                3,
                5.,
                upsample_factor=100,
                index=keypoints_shape[0],
                coordinate_scale=2 ** downsample_factor,
                confidence_scale=255.0,
            )(hm)

In [ ]:
tf.keras.backend.eval(kp2)

In [ ]:
13.5*4